In [6]:
## Import the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import os
import imblearn
from imblearn.pipeline import make_pipeline as make_pipeline_imbfinal 
from imblearn.over_sampling import SMOTE
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
#from sklearn import svm
from sklearn.metrics import *
import zipfile
from zipfile import ZipFile
from io import BytesIO
import sys
import glob
import datetime
import time
import boto.s3
from boto.s3.key import Key 
import pickle
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [3]:
df = pd.read_csv('ckd.csv')

In [7]:
df_train,df_test = train_test_split(df,train_size=0.7,random_state=42)
x_train=df_train.iloc[:,:25]
y_train=df_train['Classification_ckd']
scaler.fit(x_train)
x_train_sc=scaler.transform(x_train)
x_test=df_test.iloc[:,:25]
y_test=df_test['Classification_ckd']
scaler.fit(x_test)
x_test_sc=scaler.transform(x_test)

In [8]:
accuracy =[]
model_name =[]
dataset=[]
f1score = []
precision = []
recall = []
true_positive =[]
false_positive =[]
true_negative =[]
false_negative =[]

In [9]:
logreg = LogisticRegression()
## fitiing the model
logreg.fit(x_train_sc, y_train)
filename = 'logReg_model.sav'
pickle.dump(logreg,open(filename,'wb'))
logreg

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [10]:
prediction = logreg.predict(x_train_sc)
f1 = f1_score(y_train, prediction)
p = precision_score(y_train, prediction)
r = recall_score(y_train, prediction)
a = accuracy_score(y_train, prediction)
cm = confusion_matrix(y_train, prediction)
tp = cm[0][0]
fp = cm[0][1]
fn = cm[1][0]
tn = cm[1][1]
dataset.append('Training')
model_name.append('Logistic Regression')
f1score.append(f1)
precision.append(p)
recall.append(r)
accuracy.append(a)
true_positive.append(tp) 
false_positive.append(fp)
true_negative.append(tn) 
false_negative.append(fn)
cm

array([[106,   0],
       [  0, 174]], dtype=int64)

In [16]:
feature_list = list(x_train.columns)
importances = list(logreg.coef_)
print(importances)
print(x_train.columns)

[array([ -4.90666640e-02,   1.67724110e-01,  -7.91277042e-01,
         3.55012116e-01,   1.68671972e-01,   1.87201628e-01,
        -2.56380727e-03,   1.69428845e-01,  -2.31176549e-01,
        -5.21177094e-02,  -6.12715497e-01,  -3.65143695e-01,
         1.56530831e-01,  -2.93827549e-01,  -1.68102433e-01,
         9.83814733e-03,  -5.15959019e-02,  -1.31888878e-02,
         2.36703044e-01,   3.35081906e-01,  -1.18497482e-01,
        -2.33428375e-01,   1.73817229e-01,  -2.42457606e-02,
         2.78597709e+00])]
Index(['age', 'bp', 'sg', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hemo',
       'pcv', 'wc', 'rc', 'RBC_normal', 'PC_normal', 'PCC_present',
       'BA_present', 'HTN_yes', 'DM_yes', 'CAD_yes', 'Appet_good', 'PE_yes',
       'Ane_yes', 'Classification_ckd'],
      dtype='object')


In [17]:
rfc = RandomForestClassifier(n_estimators=50,random_state=0)
## fitiing the model
rfc.fit(x_train_sc, y_train)
filename = 'RFC_model.sav'
pickle.dump(rfc,open(filename,'wb'))
rfc

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [18]:
prediction = rfc.predict(x_train_sc)
f1 = f1_score(y_train, prediction)
p = precision_score(y_train, prediction)
r = recall_score(y_train, prediction)
a = accuracy_score(y_train, prediction)
cm = confusion_matrix(y_train, prediction)
tp = cm[0][0]
fp = cm[0][1]
fn = cm[1][0]
tn = cm[1][1]
model_name.append('Random Forest Classifier')
dataset.append('Training')
f1score.append(f1)
precision.append(p)
recall.append(r)
accuracy.append(a)
true_positive.append(tp) 
false_positive.append(fp)
true_negative.append(tn) 
false_negative.append(fn)
cm

array([[106,   0],
       [  0, 174]], dtype=int64)

In [20]:
feature_list = list(x_train.columns)
importances = list(rfc.feature_importances_)
feature_importances = [(x_train, round(importance, 2)) for x_train, importance in zip(feature_list, importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: Classification_ckd   Importance: 0.3
Variable: hemo                 Importance: 0.21
Variable: pcv                  Importance: 0.12
Variable: sc                   Importance: 0.08
Variable: rc                   Importance: 0.08
Variable: al                   Importance: 0.05
Variable: sg                   Importance: 0.04
Variable: sod                  Importance: 0.03
Variable: HTN_yes              Importance: 0.02
Variable: DM_yes               Importance: 0.02
Variable: bp                   Importance: 0.01
Variable: su                   Importance: 0.01
Variable: bgr                  Importance: 0.01
Variable: bu                   Importance: 0.01
Variable: age                  Importance: 0.0
Variable: pot                  Importance: 0.0
Variable: wc                   Importance: 0.0
Variable: RBC_normal           Importance: 0.0
Variable: PC_normal            Importance: 0.0
Variable: PCC_present          Importance: 0.0
Variable: BA_present           Importance: 0.0


In [30]:
drop_col_list=['age','pot','wc','RBC_normal','PC_normal',
               'PCC_present','BA_present','CAD_yes','Appet_good','PE_yes',
               'Ane_yes' , 'bp' , 'bu' , 'su']
x_train=df_train.iloc[:,:25]
x_train.drop(drop_col_list,axis=1,inplace=True)
y_train=df_train['Classification_ckd']
scaler.fit(x_train)
x_train_sc=scaler.transform(x_train)
x_test=df_test.iloc[:,:25]
x_test.drop(drop_col_list,axis=1,inplace=True)
y_test=df_test['Classification_ckd']
scaler.fit(x_test)
x_test_sc=scaler.transform(x_test)

In [31]:
rfc = RandomForestClassifier(n_estimators=50,random_state=0)
## fitiing the model
rfc.fit(x_train_sc, y_train)
filename = 'RFC_model.sav'
pickle.dump(rfc,open(filename,'wb'))
rfc

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [32]:
prediction = rfc.predict(x_train_sc)
f1 = f1_score(y_train, prediction)
p = precision_score(y_train, prediction)
r = recall_score(y_train, prediction)
a = accuracy_score(y_train, prediction)
cm = confusion_matrix(y_train, prediction)
tp = cm[0][0]
fp = cm[0][1]
fn = cm[1][0]
tn = cm[1][1]
model_name.append('Random Forest Classifier')
dataset.append('Training')
f1score.append(f1)
precision.append(p)
recall.append(r)
accuracy.append(a)
true_positive.append(tp) 
false_positive.append(fp)
true_negative.append(tn) 
false_negative.append(fn)
cm

array([[106,   0],
       [  0, 174]], dtype=int64)

In [33]:
feature_list = list(x_train.columns)
importances = list(rfc.feature_importances_)
feature_importances = [(x_train, round(importance, 2)) for x_train, importance in zip(feature_list, importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: Classification_ckd   Importance: 0.4
Variable: hemo                 Importance: 0.15
Variable: pcv                  Importance: 0.14
Variable: sg                   Importance: 0.09
Variable: sc                   Importance: 0.07
Variable: rc                   Importance: 0.05
Variable: DM_yes               Importance: 0.04
Variable: al                   Importance: 0.03
Variable: bgr                  Importance: 0.01
Variable: sod                  Importance: 0.01
Variable: HTN_yes              Importance: 0.01
